In [21]:
import pandas as pd

In [3]:
from BatteryProject.ModelOne import trainer
from BatteryProject.ModelOne import model_params

In [4]:
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [5]:
features = model_params.features
features = features['feature_one']

In [6]:
features

{'disc_capa': 'summary_discharge_capacity.csv',
 'dis_ener': 'summary_discharge_energy.csv',
 'temp_avg': 'summary_temperature_average.csv',
 'char_capa': 'summary_charge_capacity.csv'}

In [7]:
model_1 = model_params.models['log'][0]
grid_1 = model_params.models['log'][1]

In [10]:
trainer_1 = trainer.Trainer()

In [11]:
trainer_1.features = features

In [12]:
trainer_1.get_data(trainer_1.features)

In [13]:
trainer_1.set_pipeline(model = model_1)

In [14]:
trainer_1.run(grid_1)

In [15]:
list(trainer_1.features_name.keys())

['disc_capa', 'dis_ener', 'temp_avg', 'char_capa']

In [16]:
trainer_1.grid_search.best_params_

{'model__max_iter': 500, 'model__penalty': 'none', 'model__solver': 'lbfgs'}

In [17]:
trainer_1.eval()

{'accuracy': 0.9444444444444444,
 'precision': 0.9565217391304348,
 'roc_auc': 0.939799331103679}

In [23]:
path = '/home/romainj/code/RomainJupille/wagon/Projet_batteries/BatteryProject/BatteryProject/ModelOne/Models/Models_records.csv'

In [26]:
df_ids = pd.read_csv(path)

In [30]:
df_ids 

,Try_ID


In [31]:
df_ids = pd.read_csv(path)
if df_ids.shape[0] == 0:
    new_id=1
else:
    last_id = df_ids['Try_ID'].values.max()
    new_id = last_id + 1

In [32]:
new_id

1

In [43]:
data = {}
data['Try_ID'] = new_id
data['Model'] = trainer_1.model_name
data['Scaler'] = trainer_1.scaler_name
data['Feature_list'] = list(trainer_1.features_name.keys())
for key, value in trainer_1.evaluation.items():
    data[f"metrics_{key}"] = value
for key, values in trainer_1.grid_search.best_params_.items():
    data[f"HyperParams_{key.split('__')[1]}"] = values

In [44]:
data

{'Try_ID': 1,
 'Model': 'LogisticRegression',
 'Scaler': 'RobustScaler',
 'Feature_list': ['disc_capa', 'dis_ener', 'temp_avg', 'char_capa'],
 'metrics_accuracy': 0.9444444444444444,
 'metrics_precision': 0.9565217391304348,
 'metrics_roc_auc': 0.939799331103679,
 'HyperParams_max_iter': 500,
 'HyperParams_penalty': 'none',
 'HyperParams_solver': 'lbfgs'}

In [53]:
df_ids = df_ids.append(data, ignore_index=True)

In [55]:
df_ids.to_csv(path)

In [47]:
model_2 = model_params.models['random_forest'][0]
grid_2 = model_params.models['random_forest'][1]

trainer_2 = trainer.Trainer()
trainer_2.features = features
trainer_2.get_data(trainer_1.features)
trainer_2.set_pipeline(model = model_2)
trainer_2.run(grid_2)

In [48]:
trainer_2.grid_search.best_params_

{'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__n_estimators': 50}

In [49]:
trainer_2.eval()

{'accuracy': 0.9722222222222222,
 'precision': 0.9565217391304348,
 'roc_auc': 0.9642857142857143}

In [56]:
df_ids = pd.read_csv(path)
if df_ids.shape[0] == 0:
    new_id=1
else:
    last_id = df_ids['Try_ID'].values.max()
    new_id = last_id + 1

In [58]:
data = {}
data['Try_ID'] = new_id
data['Model'] = trainer_2.model_name
data['Scaler'] = trainer_2.scaler_name
data['Feature_list'] = list(trainer_2.features_name.keys())
for key, value in trainer_2.evaluation.items():
    data[f"metrics_{key}"] = value
for key, values in trainer_2.grid_search.best_params_.items():
    data[f"HyperParams_{key.split('__')[1]}"] = values

In [59]:
data

{'Try_ID': 2,
 'Model': 'RandomForestClassifier',
 'Scaler': 'RobustScaler',
 'Feature_list': ['disc_capa', 'dis_ener', 'temp_avg', 'char_capa'],
 'metrics_accuracy': 0.9722222222222222,
 'metrics_precision': 0.9565217391304348,
 'metrics_roc_auc': 0.9642857142857143,
 'HyperParams_min_samples_leaf': 1,
 'HyperParams_min_samples_split': 2,
 'HyperParams_n_estimators': 50}

In [67]:
df_ids = df_ids.drop(columns='Unnamed: 0').append(data, ignore_index= True)

In [ ]:
col_list = df_ids.columns
col_l1 = ['Try_ID',  'Model', 'Scaler', 'Feature_list']
col_l2 = [a for a in col_list if a[0:7] == 'metrics']
col_l3 = [a for a in col_list if a[0:11] == 'HyperParams']

In [88]:
new_col_list = col_l1 + col_l2 + col_l3

In [91]:
df_ids = df_ids[new_col_list]

In [92]:
df_ids

,Try_ID,Model,Scaler,Feature_list,metrics_accuracy,metrics_precision,metrics_roc_auc,HyperParams_max_iter,HyperParams_penalty,HyperParams_solver,HyperParams_min_samples_leaf,HyperParams_min_samples_split,HyperParams_n_estimators
0,1,LogisticRegression,RobustScaler,"['disc_capa', 'dis_ener', 'temp_avg', 'char_ca...",0.944444,0.956522,0.939799,500.0,none,lbfgs,NaN,NaN,NaN
1,2,RandomForestClassifier,RobustScaler,"[disc_capa, dis_ener, temp_avg, char_capa]",0.972222,0.956522,0.964286,NaN,NaN,NaN,1.0,2.0,50.0


In [93]:
df_ids.to_csv(path)